Initial data exploration to pull data. Work on cleaning and preparing for analysis.

In [147]:
# Import necessary modules
import os
import glob
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None) # Makes pandas full width with scrollable window.

In [148]:
# Initialize variables for working directories where the data is located.
players_directory = './players'
shots_directory = './shots'

In [149]:
# Load the salary data into a dataframe.
salaries_df = pd.read_csv(players_directory + '/salaries_1985to2018.csv')

print(salaries_df.head(10))

   index league  player_id   salary   season  season_end  season_start                    team
0      0    NBA  abdelal01   395000  1990-91        1991          1990  Portland Trail Blazers
1      1    NBA  abdelal01   494000  1991-92        1992          1991  Portland Trail Blazers
2      2    NBA  abdelal01   500000  1992-93        1993          1992          Boston Celtics
3      3    NBA  abdelal01   805000  1993-94        1994          1993          Boston Celtics
4      4    NBA  abdelal01   650000  1994-95        1995          1994        Sacramento Kings
5      5    NBA  abdulka01  1530000  1984-85        1985          1984      Los Angeles Lakers
6      6    NBA  abdulka01  2030000  1985-86        1986          1985      Los Angeles Lakers
7      7    NBA  abdulka01  2000000  1987-88        1988          1987      Los Angeles Lakers
8      8    NBA  abdulka01  3000000  1988-89        1989          1988      Los Angeles Lakers
9      9    NBA  abdulma02  1660000  1990-91      

In [150]:
# Load the player data into a dataframe and print the head.
players_df= pd.read_csv(players_directory + '/players.csv')

print(players_df.head(10))

   index        _id          birthDate                birthPlace  career_AST career_FG% career_FG3% career_FT%  career_G career_PER  career_PTS career_TRB career_WS career_eFG%                                            college    draft_pick    draft_round              draft_team draft_year height                               highSchool                 name                                    position shoots weight
0      0  abdelal01      June 24, 1968              Cairo, Egypt         0.3       50.2         0.0       70.1       256       13.0         5.7        3.3       4.8        50.2                                    Duke University  25th overall      1st round  Portland Trail Blazers       1990   6-10     Bloomfield in Bloomfield, New Jersey       Alaa Abdelnaby                               Power Forward  Right  240lb
1      1  abdulza01      April 7, 1946        Brooklyn, New York         1.2       42.8         NaN       72.8       505       15.1         9.0        8.0      17

In [151]:
# Load the shots data for 2024 into a dataframe and print the head.
shots_2024_df = pd.read_csv(shots_directory + '/NBA_2024_Shots.csv')

print(shots_2024_df.head(10))

   SEASON_1 SEASON_2     TEAM_ID           TEAM_NAME  PLAYER_ID   PLAYER_NAME POSITION_GROUP POSITION   GAME_DATE   GAME_ID HOME_TEAM AWAY_TEAM   EVENT_TYPE  SHOT_MADE                     ACTION_TYPE       SHOT_TYPE             BASIC_ZONE          ZONE_NAME ZONE_ABB       ZONE_RANGE  LOC_X  LOC_Y  SHOT_DISTANCE  QUARTER  MINS_LEFT  SECS_LEFT
0      2024  2023-24  1610612764  Washington Wizards    1629673  Jordan Poole              G       SG  11-03-2023  22300003       MIA       WAS  Missed Shot      False      Driving Floating Jump Shot  2PT Field Goal  In The Paint (Non-RA)             Center        C         8-16 ft.   -0.4  17.45             12        1         11          1
1      2024  2023-24  1610612764  Washington Wizards    1630166   Deni Avdija              F       SF  11-03-2023  22300003       MIA       WAS    Made Shot       True                       Jump Shot  3PT Field Goal      Above the Break 3             Center        C          24+ ft.    1.5  30.55             25

As seen in the dataframe heads above, the 'salaries_1985to2018.csv' and 'players.csv' are joinable on the 'player_id' and '_id' columns in each file respectively. This makes sense given that these files are both from the same dataset.

Using the 'NBA_2024_Shots.csv' as an example from our shots directory, we see the need to create an id column that can be used to join the data.

The id columns in the 'salaries_1985to2018.csv' and 'players.csv' appear to have been generated from the first five characters of the last name, first two characters of the first name, and a calculated number starting at 01, incrementing up if there are duplications due to players having similar names.

We will define a function that will perform this same action on our shots data.

In [152]:
# First, we need to split the 'PLAYER_NAME' column to separate the first and last names.
shots_2024_df[['first_name', 'last_name']] = shots_2024_df['PLAYER_NAME'].str.split(' ', n=1, expand=True)

In [153]:
#Print the head to see the new columns generated from the existing 'PLAYER_NAME' column.
print(shots_2024_df[['PLAYER_NAME', 'first_name', 'last_name']].head(10))

    PLAYER_NAME first_name last_name
0  Jordan Poole     Jordan     Poole
1   Deni Avdija       Deni    Avdija
2    Tyus Jones       Tyus     Jones
3  Jordan Poole     Jordan     Poole
4    Tyus Jones       Tyus     Jones
5    Kyle Kuzma       Kyle     Kuzma
6    Tyus Jones       Tyus     Jones
7  Jordan Poole     Jordan     Poole
8   Deni Avdija       Deni    Avdija
9   Deni Avdija       Deni    Avdija


In [154]:
# Dictionary to store PLAYER_ID mapped to new_player_id, to only create a new id if the player is truly a one off where the id generated is a dup of another player.
player_id_map = {}

# Create a function to generate new_player_id from first name, last name, and a dynamic incremented value to deal with duplicates.
def generate_new_player_id(row, player_id_counts):
    player_id = row['PLAYER_ID']  # Use the existing PLAYER_ID for consistency
    
    # Check if we've already generated a new_player_id for this PLAYER_ID
    if player_id in player_id_map:
        return player_id_map[player_id]
    
    last_name = row['last_name'].lower()[:5]  # Get the first 5 letters of last name.
    first_name = row['first_name'].lower()[:2]  # Get the first 2 letters of first name.
    base_name_id = last_name + first_name
    
    # Check if the base_name_id already exists, if it does increment the number that will be appended
    if base_name_id in player_id_counts:
        player_id_counts[base_name_id] += 1
    else:
        player_id_counts[base_name_id] = 1
    
    # Format the number as two digits (01, 02, 03, 04, etc...)
    new_player_id = base_name_id + f"{player_id_counts[base_name_id]:02d}"
    
    # Save the generated new_player_id for future use
    player_id_map[player_id] = new_player_id
    
    return new_player_id

In [155]:
# Create a dictionary to keep track of counts for each base_name_id
player_id_counts = {}

# Apply the function to create the new_player_id column
shots_2024_df['new_player_id'] = shots_2024_df.apply(generate_new_player_id, axis=1, player_id_counts=player_id_counts)

# Print the head to see the new_player_id generated from the existing columns.
print(shots_2024_df[['PLAYER_NAME', 'PLAYER_ID', 'first_name', 'last_name', 'new_player_id']].head(10))

    PLAYER_NAME  PLAYER_ID first_name last_name new_player_id
0  Jordan Poole    1629673     Jordan     Poole     poolejo01
1   Deni Avdija    1630166       Deni    Avdija     avdijde01
2    Tyus Jones    1626145       Tyus     Jones     jonesty01
3  Jordan Poole    1629673     Jordan     Poole     poolejo01
4    Tyus Jones    1626145       Tyus     Jones     jonesty01
5    Kyle Kuzma    1628398       Kyle     Kuzma     kuzmaky01
6    Tyus Jones    1626145       Tyus     Jones     jonesty01
7  Jordan Poole    1629673     Jordan     Poole     poolejo01
8   Deni Avdija    1630166       Deni    Avdija     avdijde01
9   Deni Avdija    1630166       Deni    Avdija     avdijde01


In [156]:
# Now that we have the new_player_id that should match the ids from the salaries_df and players_df, we can merge the dataframes.

#Todo: Swap to 2018 shots, realizing now that salaries data only has up to 2018. Then, isolate the salaries for 2018, we only want to explore that single year for this initial exploration.
# Once that is completed, we should be able to merge the dataframes on records with matching player ids.